In [1]:
'''was trying to copy a tutorial but I got pretty stumped 
changig the output to something catergorical
https://www.tensorflow.org/tutorials/text/text_generation
'''

'was trying to copy a tutorial but I got pretty stumped \nchangig the output to something catergorical\nhttps://www.tensorflow.org/tutorials/text/text_generation\n'

In [2]:
import tensorflow as tf
import numpy as np
import math
import random

In [3]:
tf.compat.v1.enable_eager_execution()

In [4]:
tf.__version__
tf.executing_eagerly()

True

In [5]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [6]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [7]:
# Take a look at the first 250 characters in text
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [8]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

65 unique characters


In [9]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [10]:
print('{')
for char,_ in zip(char2idx, range(5)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  ...
}


In [11]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [12]:
# The maximum length sentence we want for a single input in characters
seq_length = 32
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

F
i
r
s
t


In [13]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [14]:
def assign_radom_n(chunk):
    input_text = chunk[:-1]
    target_n = 1 #random.randint(1,41)
    return input_text, target_n

dataset = sequences.map(assign_radom_n)

In [15]:
for input_example, target_n in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(str(target_n.numpy())))

Input data:  'First Citizen:\nBefore we proceed'
Target data: '1'


In [16]:
# Batch size
BATCH_SIZE = 128

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 1000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [24]:
dataset

<DatasetV1Adapter shapes: ((128, 32), (128,)), types: (tf.int64, tf.int32)>

In [33]:
def build_model2(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, 32]),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10000),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(2500),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(128),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(40),
    tf.keras.layers.Softmax(1)
  ])
    
    return model

In [28]:
def build_model(vocab_size, embedding_dim, batch_size):
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, 32]),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128),
    tf.keras.layers.Dense(40, activation='softmax')
    ])
    
    return model
    

In [29]:
# Length of the vocabulary in chars

vocab_size = len(vocab)
embedding_dim = 12
rnn_units = 200

model = build_model(
  vocab_size = len(vocab),
  embedding_dim= embedding_dim,
#   rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [30]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length,number_of_cats)")

(128, 40) # (batch_size, sequence_length,number_of_cats)


In [31]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (128, 32, 12)             780       
_________________________________________________________________
flatten_1 (Flatten)          (128, 384)                0         
_________________________________________________________________
dense_2 (Dense)              (128, 128)                49280     
_________________________________________________________________
dense_3 (Dense)              (128, 40)                 5160      
Total params: 55,220
Trainable params: 55,220
Non-trainable params: 0
_________________________________________________________________


In [32]:
row = dataset.take(1)
temp = model.predict(row)[0]
print(len(temp)), temp


40


(None, array([0.02430696, 0.0248217 , 0.02477421, 0.02630217, 0.02453539,
        0.02665857, 0.02463816, 0.02589303, 0.02498468, 0.02420962,
        0.02506692, 0.02479428, 0.02427546, 0.02529157, 0.02409277,
        0.02596217, 0.02583934, 0.02535408, 0.02405896, 0.02320016,
        0.02442126, 0.02648077, 0.02512112, 0.02523943, 0.02422761,
        0.02465073, 0.02615336, 0.0254291 , 0.02447377, 0.0244334 ,
        0.02275205, 0.02558434, 0.02679243, 0.02448179, 0.02484056,
        0.02479533, 0.02600506, 0.02560321, 0.02422371, 0.0252308 ],
       dtype=float32))

In [33]:
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [34]:
EPOCHS=20
history = model.fit(dataset, epochs=EPOCHS)

Epoch 1/20
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
264/264 [==============================] - 4s 16ms/step - loss: 147.5564
Epoch 2/20
264/264 [==============================] - 4s 14ms/step - loss: 147.5552
Epoch 3/20
264/264 [==============================] - 4s 14ms/step - loss: 147.5552
Epoch 4/20
264/264 [==============================] - 4s 14ms/step - loss: 147.5552
Epoch 5/20
264/264 [==============================] - 4s 14ms/step - loss: 147.5552
Epoch 6/20
264/264 [==============================] - 4s 14ms/step - loss: 147.5552
Epoch 7/20
165/264 [=================>............] - ETA: 1s - loss: 147.5552

KeyboardInterrupt: 